In [1]:
import os 
import glob
from ase.io import read,write

# find Ovito Files --> Define Folder

In [2]:
simulation_folder = '/nfshome/winkelmann/running_sims/Lichtenberg_X-mas_25_26'


pattern = os.path.join(simulation_folder, '**', '*.XDATCAR')
path_all_ovito_files = sorted(glob.glob(pattern, recursive=True))

# Find all truncated files and create list of their original files
truncated_files = [f for f in path_all_ovito_files if f.endswith('_truncated.XDATCAR')]
original_files_to_exclude = [f.replace('_truncated.XDATCAR', '.XDATCAR') for f in truncated_files]

# Exclude both truncated files and their base files
path_all_ovito_files = [f for f in path_all_ovito_files if f not in truncated_files and f not in original_files_to_exclude]
print(f"Found {len(path_all_ovito_files)} ovito files to process.")
print(f'excluded {len(truncated_files)} truncated files and their base files.')
print('\n'.join(f'{i+1}. {f}' for i, f in enumerate(sorted(original_files_to_exclude))))

Found 180 ovito files to process.
excluded 0 truncated files and their base files.



# truncate to only Li and X Atoms

In [4]:
def truncate_ovito_files(path_ovito_file, wanted_species=['Li','X']):
    all_frames = read(path_ovito_file, index=':')
    truncated_ovito_file = []
    for atoms in all_frames:  
        indices = [i for i, atom in enumerate(atoms) if atom.symbol in wanted_species]
        truncated_ovito_file.append(atoms[indices])      
    write(path_ovito_file.replace('.XDATCAR', '_truncated.XDATCAR'), truncated_ovito_file)
    return truncated_ovito_file

for i, path_ovito_file in enumerate(path_all_ovito_files):
    progress = (i + 1) / len(path_all_ovito_files) * 100
    print(f'Progress: {i/len(path_all_ovito_files) * 100:.0f}% \t Processing File {i+1}/{len(path_all_ovito_files)}', end='\r', flush=True)
    truncate_ovito_files(path_ovito_file)

# find trunicated files ()

# calculate Warren Cowley and safe it to a CSV